In [63]:
# !pip install gym-trading-env
import numpy as np
import pandas as pd
import datetime

from gym_trading_env.downloader import download
from statsmodels.tsa.stattools import acf, pacf
from sklearn.linear_model import LinearRegression
import ml_metrics as metrix

# import matplotlib.pyplot as plt
# %matplotlib inline
# from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
# import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode, iplot
# import seaborn as sns

import warnings
warnings.simplefilter('ignore')

# Nostradamus  MODEL 1Hour Prediction

In [ ]:
################DOWNLOADER################

In [75]:
# df.to_pickle('df_.pkl')
df = pd.read_pickle('df_.pkl')
df

,open,high,low,close,volume,date_close,date_open,VWAP,seasonal_index,trend_index,resid_index,UnixH
date_close,,,,,,,,,,,,
2020-02-10 00:00:00,10126.22,10160.00,10119.78,10151.75,1786.197512,2020-02-10 00:00:00,2020-02-09 23:00:00,10151.750000,1.171449,3946.090128,2.196092,439248.0
2020-02-10 01:00:00,10151.72,10188.00,10150.00,10184.19,1767.194315,2020-02-10 01:00:00,2020-02-10 00:00:00,10167.883257,1.172082,3946.677363,2.198067,439249.0
2020-02-10 02:00:00,10184.19,10184.40,10130.00,10133.32,1151.821611,2020-02-10 02:00:00,2020-02-10 01:00:00,10159.422279,1.171743,3947.264599,2.196547,439250.0
2020-02-10 03:00:00,10133.33,10147.26,10080.00,10120.59,2238.188839,2020-02-10 03:00:00,2020-02-10 02:00:00,10146.904788,1.171206,3947.851834,2.194520,439251.0
2020-02-10 04:00:00,10120.40,10120.40,10060.00,10090.47,1750.261272,2020-02-10 04:00:00,2020-02-10 03:00:00,10135.542993,1.170725,3948.439069,2.192636,439252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-11 07:00:00,48224.71,48314.54,48125.00,48221.94,1128.067970,2024-02-11 07:00:00,2024-02-11 06:00:00,25679.234330,1.161545,25200.883263,0.877264,474343.0
2024-02-11 08:00:00,48221.94,48355.00,48061.10,48179.52,1037.190030,2024-02-11 08:00:00,2024-02-11 07:00:00,25679.399930,1.161352,25200.944076,0.877413,474344.0
2024-02-11 09:00:00,48179.51,48286.55,48075.87,48140.00,800.501470,2024-02-11 09:00:00,2024-02-11 08:00:00,25679.527513,1.161153,25201.004931,0.877566,474345.0


In [78]:
# Определяем количество элементов вручную которые влияют на наш прогноз
p = 35069 # p место с которого мы ничего не знаем для теста и поиска (последнее значение подаваемой выборки)
df.head(p)


,open,high,low,close,volume,date_close,date_open,VWAP,seasonal_index,trend_index,resid_index,UnixH
date_close,,,,,,,,,,,,
2020-02-10 00:00:00,10126.22,10160.00,10119.78,10151.75,1786.197512,2020-02-10 00:00:00,2020-02-09 23:00:00,10151.750000,1.171449,3946.090128,2.196092,439248.0
2020-02-10 01:00:00,10151.72,10188.00,10150.00,10184.19,1767.194315,2020-02-10 01:00:00,2020-02-10 00:00:00,10167.883257,1.172082,3946.677363,2.198067,439249.0
2020-02-10 02:00:00,10184.19,10184.40,10130.00,10133.32,1151.821611,2020-02-10 02:00:00,2020-02-10 01:00:00,10159.422279,1.171743,3947.264599,2.196547,439250.0
2020-02-10 03:00:00,10133.33,10147.26,10080.00,10120.59,2238.188839,2020-02-10 03:00:00,2020-02-10 02:00:00,10146.904788,1.171206,3947.851834,2.194520,439251.0
2020-02-10 04:00:00,10120.40,10120.40,10060.00,10090.47,1750.261272,2020-02-10 04:00:00,2020-02-10 03:00:00,10135.542993,1.170725,3948.439069,2.192636,439252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-11 07:00:00,48224.71,48314.54,48125.00,48221.94,1128.067970,2024-02-11 07:00:00,2024-02-11 06:00:00,25679.234330,1.161545,25200.883263,0.877264,474343.0
2024-02-11 08:00:00,48221.94,48355.00,48061.10,48179.52,1037.190030,2024-02-11 08:00:00,2024-02-11 07:00:00,25679.399930,1.161352,25200.944076,0.877413,474344.0
2024-02-11 09:00:00,48179.51,48286.55,48075.87,48140.00,800.501470,2024-02-11 09:00:00,2024-02-11 08:00:00,25679.527513,1.161153,25201.004931,0.877566,474345.0


In [83]:
%%time
acf_value= acf(df.close.head(p))
pacf_value = pacf(df.close.head(p))

y = acf_value[1:].tolist()
y_= pacf_value[1:].tolist()
y.reverse()
y_.reverse()
z=len(y)

x=[]
for i in range(0,z):
    x.append(i)
    
close1 = df.close[(p-z):p]


data= pd.DataFrame({'x':x, 'y':y, 'y_':y_,'close':close1})
data['modeling']=data.y*data.close
data['modeling_']=data.modeling*data.y_

zn = 45
# plt.plot(data.x.tail(zn),data.modeling.tail(zn))  #визуализация для лохов

#Последние два значения перед прогнозируемым
x_train = data.x.tail(zn).head(zn-1)
y_train = data.modeling.tail(zn).head(zn-1)
# y_test = df.close[p:(p+1)].item()
x_test=pd.DataFrame({45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80})
# x_test

lr=LinearRegression().fit(x_train.values.reshape(-1,1), y_train.values.reshape(-1,1))
pred= lr.predict(x_test.values.reshape(-1,1))

delta = abs(y_train[-1]-pred)
deltasqr=abs(y_train[-1]-pred)
deltadf = pd.DataFrame({'delta':delta.flatten(), 'pred':pred.flatten()})
deltadf

CPU times: total: 1.09 s
Wall time: 331 ms


,delta,pred
0,98.560403,48024.118793
1,68.886830,48053.792366
2,39.213258,48083.465938
3,9.539685,48113.139511
4,20.133888,48142.813084
5,49.807460,48172.486656
6,79.481033,48202.160229
7,109.154606,48231.833802
8,138.828178,48261.507375
9,168.501751,48291.180947


2.856805503644864
2.856805503644864
precision=[99.99406597]
y_test = 48142.78, pred = [[48139.9231945]]     НАСТРОЕННОЕ ПЕРВИЧНОЕ ЗНАЧЕНИЕ

79.49877820649272
79.49877820649272
precision=[99.83359485]
y_test = 47774.23, pred = [[47694.73122179]]     ТЕСТОВОЕ ЗНАЧЕНИЕ zn = 45    при дельта 20 можно использовать прогноз

In [ ]:
73.76714365826774
73.76714365826774
precision=[99.84559219]
y_test = 47774.23, pred = [[47700.46285634]]    zn=45

протестировать на другой выборке
ЧТО ЕСЛИ ИЗЗА ТОГО ЧТО Я ДАЛ ЗНАЧЕНИЕ ГОТОВОЕ В ACF   


добавить для точности PACF